# Monthly Forecasting Model

## This forecasting model does the following:
 1. Runs several models on training data to determine the most accurate forecast
 2. Builds out a two-year forecast using the best forecast model
 3. Graphs the forecast with confidence intervals
 4. Returns a data export to excel

### (i) - Import Packages
Imports required packages for modeling, analysis, and graphing

In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import stdev
from statsmodels.tsa.api import ExponentialSmoothing
from pmdarima import auto_arima
from dateutil.relativedelta import *

### (ii) - Organize data
Splits data into various DataFrames by heirarchy

In [ ]:
def data_split(df):
    """
    Splits a 3-hierarchy DataFrame into seperate forecasting DataFrames
    
    Arguments:
    df -- DataFrame to be split
    
    Returns:
    top_df -- DataFrame for the top 'total' hierarchy
    mid_df -- DataFrame for the middle hierarchy
    mid_list -- list of each value in the middle hierarchy
    mid_cnt -- count of values in the middle hierarchy
    low_dfs -- dictionary containing all lower DataFrames
    low_lists -- dictionary containing all lower values
    low_cnts -- dictionary containing all lower value counts
    """
    
    # Top df
    top_df = df.groupby(df.columns[2]).sum().rename(columns={df.columns[3]: 'total'})

    # Mid df
    mid_df = df.groupby([df.columns[0],df.columns[2]], as_index=False).sum()
    mid_list = mid_df[df.columns[0]].unique()
    mid_cnt = mid_df[df.columns[0]].nunique()
    mid_df = mid_df.pivot(index = df.columns[2], columns = df.columns[0], values = df.columns[3])
    mid_df = pd.concat([top_df, mid_df], axis=1, sort=False)

    # Bottom dfs
    low_df = df.groupby([df.columns[0],df.columns[1],df.columns[2]], as_index=False).sum()
    low_dfs = {}
    low_lists = {}
    low_cnts = {}

    for i in range(0,mid_cnt):

        low_dfi = low_df.copy()
        low_dfi = low_dfi[low_dfi[low_dfi.columns[0]] == mid_list[i]]
        low_lists["low_list" + str(i)] = low_dfi[low_dfi.columns[1]].unique()
        low_cnts["low_cnt" + str(i)] = low_dfi[low_dfi.columns[1]].nunique()

        low_dfi = low_dfi.pivot(index = low_dfi.columns[2], columns = low_dfi.columns[1], values = df.columns[3])
        low_dfs["low_df" + str(i)] = pd.concat([mid_df.iloc[:,[i+1]], low_dfi], axis=1, sort=False)
    
    return top_df, mid_df, mid_list, mid_cnt, low_dfs, low_lists, low_cnts

### 1.1 - Training Model
Creates a training and test set from the data for out-of-sample testing

In [234]:
def train_model(variable):
    """
    Creates a training and test set from the data.
    
    Arguments:
    variable -- array to be forecasted
    
    Returns:
    trainset -- dataset with the testset removed
    testset -- dataset used for testing forecast accuracy
    testsize -- int showing the size of the testset
    """
    
    #calculate testsize:
    if variable.index.size >=36:
        testsize = 12
    else:
        testsize = int(36 - variable.index.size)
        
    ##test periods override:
    testsize = 6
    
    #create training/learning sets
    trainset = variable[0:variable.index.size - testsize]
    testset = variable[variable.index.size - testsize:]
    
    return trainset, testset, testsize

### 1.2 - Testing Model
Forecasts the 12 Holt-Winters exponential smoothing models and seasonal auto-ARIMA model

In [235]:
def test_model(trainset, testset, testsize):
    """
    Forecasts the 12 Holt-Winters exponential smoothing models and seasonal auto-ARIMA model
    
    Arguments:
    trainset -- DataFrame to be forecasted
    testset -- DataFrame for forecast to be tested against
    testsize -- size of the testset (int)
    
    Returns:
    fcst_results -- DataFrame containing all forecast results
    
    """
    fcst_results = testset.copy()
    
    #Run the 12 Exponential Smoothing (Holt-Winters) forecasting models
    try:
        
        fit_MN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='mul', damped=False, seasonal=None,).fit()
        fit_MA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='mul', damped=False, seasonal='add',).fit()
        fit_MM = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='mul', damped=False, seasonal='mul',).fit()
        fit_NN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend=None, damped=False, seasonal=None,).fit()
        fit_NA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend=None, damped=False, seasonal='add',).fit()
        fit_NM = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend=None, damped=False, seasonal='mul',).fit()
        fit_LN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=False, seasonal=None,).fit()
        fit_LA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=False, seasonal='add',).fit()
        fit_LM = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=False, seasonal='mul',).fit()
        fit_DN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=True, seasonal=None,).fit()
        fit_DA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=True, seasonal='add',).fit()
        fit_DM = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=True, seasonal='mul',).fit()
        
        #Create a table of all the forecasts
        fcst_results['NM_y_hat'] = fit_NM.forecast(testsize)
        fcst_results['LM_y_hat'] = fit_LM.forecast(testsize)
        fcst_results['DM_y_hat'] = fit_DM.forecast(testsize)
        fcst_results['MN_y_hat'] = fit_MN.forecast(testsize)
        fcst_results['MA_y_hat'] = fit_MA.forecast(testsize)
        fcst_results['MM_y_hat'] = fit_MM.forecast(testsize)
        
    except NotImplementedError:
        
        fit_NN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend=None, damped=False, seasonal=None,).fit()
        fit_NA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend=None, damped=False, seasonal='add',).fit()
        fit_LN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=False, seasonal=None,).fit()
        fit_LA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=False, seasonal='add',).fit()
        fit_DN = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=True, seasonal=None,).fit()
        fit_DA = ExponentialSmoothing(np.asarray(trainset) ,seasonal_periods=12 ,trend='add', damped=True, seasonal='add',).fit()
        
        #Create a table of all the forecasts
        fcst_results['NM_y_hat'] = None
        fcst_results['LM_y_hat'] = None
        fcst_results['DM_y_hat'] = None
        fcst_results['MN_y_hat'] = None
        fcst_results['MA_y_hat'] = None
        fcst_results['MM_y_hat'] = None

    finally:
        #Run the seasonal Auto-ARIMA (Box-Jenkins) forecast model
        stepwise_model = auto_arima(trainset, start_p=1, start_q=1,
                                   max_p=3, max_q=3, m=12,
                                   start_P=1, seasonal=True,
                                   error_action='ignore',
                                   suppress_warnings=True,
                                   stepwise=True)

        stepwise_model.fit(np.asarray(trainset))
        fit_ARIMA = stepwise_model.predict(testsize)
        
        #Create a table of all the forecasts
        fcst_results['NN_y_hat'] = fit_NN.forecast(testsize)
        fcst_results['NA_y_hat'] = fit_NA.forecast(testsize)
        fcst_results['LN_y_hat'] = fit_LN.forecast(testsize)
        fcst_results['LA_y_hat'] = fit_LA.forecast(testsize)
        fcst_results['DN_y_hat'] = fit_DN.forecast(testsize)
        fcst_results['DA_y_hat'] = fit_DA.forecast(testsize)
        fcst_results['ARIMA_y_hat'] = fit_ARIMA
    
        return fcst_results

### 1.3 - Accuracy Model
Calculates the error, MAPE, and standard deviation for each model and selects the best forecast

In [236]:
def acc_model(fcst_results, testsize, override=False, or_season=False):
    """
    Calculates the error, MAPE, and standard deviation for each model and selects the best forecast
    
    Arguments:
    fcst_results -- DataFrame containing forecast results
    testsize -- int showing the size of the testset
    override -- optional string, override the forecast
    or_season -- optional string, override forecast seasonality
    
    Returns:
    best_fcst -- String signifying the winning model
    fcst_error -- Percentage error for the best forecast
    bf_stddev -- Int standard deviation of the errors for the best forecast
    
    """
    
    #Caluculates the absolute error for each forecast period
    variable = fcst_results.columns[0]
    
    fcst_results['NN_E'] = abs(fcst_results[variable] - fcst_results['NN_y_hat'])
    fcst_results['NA_E'] = abs(fcst_results[variable] - fcst_results['NA_y_hat'])
    fcst_results['NM_E'] = abs(fcst_results[variable] - fcst_results['NM_y_hat'])
    fcst_results['LN_E'] = abs(fcst_results[variable] - fcst_results['LN_y_hat'])
    fcst_results['LA_E'] = abs(fcst_results[variable] - fcst_results['LA_y_hat'])
    fcst_results['LM_E'] = abs(fcst_results[variable] - fcst_results['LM_y_hat'])
    fcst_results['DN_E'] = abs(fcst_results[variable] - fcst_results['DN_y_hat'])
    fcst_results['DA_E'] = abs(fcst_results[variable] - fcst_results['DA_y_hat'])
    fcst_results['DM_E'] = abs(fcst_results[variable] - fcst_results['DM_y_hat'])
    fcst_results['MN_E'] = abs(fcst_results[variable] - fcst_results['MN_y_hat'])
    fcst_results['MA_E'] = abs(fcst_results[variable] - fcst_results['MA_y_hat'])
    fcst_results['MM_E'] = abs(fcst_results[variable] - fcst_results['MM_y_hat'])
    fcst_results['ARIMA_E'] = abs(fcst_results[variable] - fcst_results['ARIMA_y_hat'])
    
    #Caluculates Absolute Percentage Error (APE) for each forecast period
    
    fcst_results['NN_APE'] = fcst_results['NN_E'] / fcst_results[variable]
    fcst_results['NA_APE'] = fcst_results['NA_E'] / fcst_results[variable]
    fcst_results['NM_APE'] = fcst_results['NM_E'] / fcst_results[variable]
    fcst_results['LN_APE'] = fcst_results['LN_E'] / fcst_results[variable]
    fcst_results['LA_APE'] = fcst_results['LA_E'] / fcst_results[variable]
    fcst_results['LM_APE'] = fcst_results['LM_E'] / fcst_results[variable]
    fcst_results['DN_APE'] = fcst_results['DN_E'] / fcst_results[variable]
    fcst_results['DA_APE'] = fcst_results['DA_E'] / fcst_results[variable]
    fcst_results['DM_APE'] = fcst_results['DM_E'] / fcst_results[variable]
    fcst_results['MN_APE'] = fcst_results['MN_E'] / fcst_results[variable]
    fcst_results['MA_APE'] = fcst_results['MA_E'] / fcst_results[variable]
    fcst_results['MM_APE'] = fcst_results['MM_E'] / fcst_results[variable]
    fcst_results['ARIMA_APE'] = fcst_results['ARIMA_E'] / fcst_results[variable]
    
    #Calculates the Mean Absolute Percentage Error (MAPE) for each forecast

    mape = pd.DataFrame({'Model': ['NN','NA','NM','LN','LA','LM',
                                    'DN','DA','DM','MN','MA','MM','ARIMA'],

                         'Error': [sum(fcst_results['NN_E']),
                                   sum(fcst_results['NA_E']),
                                   sum(fcst_results['NM_E']),
                                   sum(fcst_results['LN_E']),
                                   sum(fcst_results['LA_E']),
                                   sum(fcst_results['LM_E']),
                                   sum(fcst_results['DN_E']),
                                   sum(fcst_results['DA_E']),
                                   sum(fcst_results['DM_E']),
                                   sum(fcst_results['MN_E']),
                                   sum(fcst_results['MA_E']),
                                   sum(fcst_results['MM_E']),
                                   sum(fcst_results['ARIMA_E'])],                               

                          'MAPE':[sum(fcst_results['NN_APE'])/testsize,
                                  sum(fcst_results['NA_APE'])/testsize,
                                  sum(fcst_results['NM_APE'])/testsize,
                                  sum(fcst_results['LN_APE'])/testsize,
                                  sum(fcst_results['LA_APE'])/testsize,
                                  sum(fcst_results['LM_APE'])/testsize,
                                  sum(fcst_results['DN_APE'])/testsize,
                                  sum(fcst_results['DA_APE'])/testsize,
                                  sum(fcst_results['DM_APE'])/testsize,
                                  sum(fcst_results['MN_APE'])/testsize,
                                  sum(fcst_results['MA_APE'])/testsize,
                                  sum(fcst_results['MM_APE'])/testsize,
                                  sum(fcst_results['ARIMA_APE'])/testsize],

                          'e_stdev':[stdev(fcst_results['NN_E']),
                                  stdev(fcst_results['NA_E']),
                                  stdev(fcst_results['NM_E']),
                                  stdev(fcst_results['LN_E']),
                                  stdev(fcst_results['LA_E']),
                                  stdev(fcst_results['LM_E']),
                                  stdev(fcst_results['DN_E']),
                                  stdev(fcst_results['DA_E']),
                                  stdev(fcst_results['DM_E']),
                                  stdev(fcst_results['MN_E']),
                                  stdev(fcst_results['MA_E']),
                                  stdev(fcst_results['MM_E']),
                                  stdev(fcst_results['ARIMA_E'])]})
    
    #Selects the winning forecast or override
    
    if override == False:
        try:
            best_fcst = mape.iloc[mape['MAPE'].idxmin(),0]
        except TypeError:
            best_fcst = mape.iloc[mape['Error'].idxmin(),0]
            
    else:
        best_fcst = override

    if or_season == False:
        pass
    else:
        if best_fcst == 'ARIMA':
            pass
        else:
            best_fcst = best_fcst[0] + or_season
            
    fcst_index = mape.index[mape['Model'] == best_fcst][0]
    fcst_error = round(mape.iloc[fcst_index,2],2)
    bf_stdev = mape.iloc[fcst_index,3]
    
    return best_fcst, fcst_error, bf_stdev

### 2.1 Forecast Model
Builds the final forecast model using the best forecast and complete dataset

In [237]:
def fcst_model(fcst_df, best_fcst, d0):
    """
    Builds the final forecast model using the best forecast and complete dataset
    
    Arguments:
    fcst_df -- DataFrame to be forecasted
    best_fcst -- String of winning forecast model (or override)
    d0 -- Date of final historical month
    
    Returns:
    history -- DataFrame containing all history
    forecast -- DataFrame containing the final forecast
    
    """
    
    #Trend
    if best_fcst[0][0] == 'N':
        trnd = None
    else:
        if best_fcst[0][0] == 'M':
            trnd = 'mul'
        else:
            trnd = 'add'

    #Dampening
    if best_fcst[0][0] == 'D':
        dmpd = True
    else:
        dmpd = False

    #Seasonality
    if best_fcst[1][0] == 'N':
        season = None
    else:
        if best_fcst[0][0] == 'M':
            season = 'mul'
        else:
            season = 'add'

    #Final Forecast
    if best_fcst == 'ARIMA':
        
        stepwise_model = auto_arima(fcst_df, start_p=1, start_q=1,
                                   max_p=3, max_q=3, m=12,
                                   start_P=1, seasonal=True,
                                   error_action='ignore',
                                   suppress_warnings=True,
                                   stepwise=True)
        
        stepwise_model.fit(np.asarray(fcst_df))
        fit_best = stepwise_model.predict(24)

    else:
        fit_best = ExponentialSmoothing(np.asarray(fcst_df), seasonal_periods=12, trend=trnd, damped=dmpd, seasonal=season,).fit()
        fit_best = fit_best.forecast(24)
    
    #Add Dates to final forecast
    fin_df = fcst_df
    variable = fcst_df.columns[0]
    
    for i in range(fit_best.size):
        next_month = d0 + relativedelta(months=+i+1)

        fcst_period = pd.DataFrame([[next_month, fit_best[i]]], columns=['Month',variable]).set_index('Month')

        fin_df = fin_df.append(fcst_period, sort=True)
    
    #Final DataFrames
    d1 = fin_df.index.max()
    
    history = fin_df[0:fcst_df.size].copy()
    forecast = fin_df[fcst_df.size:].copy()
    
    forecast[forecast.columns[0]] = [0 if i < 0 else i for i in forecast[forecast.columns[0]]]
    
    return history, forecast

### 2.2 (optional) - Topdown Model
Reallocates 'bottom' forecasts based on percentage of total 'top' forecast

In [238]:
def allocate_fcst(data, prev_fcst, error, top_column, bot_first, bot_last, topdown=False):
    """
    Topdown = False: Forces top forecast to equal the sum of bottom forecasts
    Topdown = True: Reallocates 'bottom' forecasts based on percentage of total 'top' forecast
    
    Arguments:
    data -- DataFrame to be reallocated
    prev_fcst -- DataFrame of origional testing/training forecast
    error -- DataFrame showing the MAPE, stdev, and model for the origional forecast
    top_column -- Column that contains the top forecast
    bot_first -- first column of 'bottom' forecast range
    bot_last: -- last column of 'bottom' forecast range (must be in order)
    
    Returns:
    fin_data -- DataFrame altered to contain the reallocated forecast
    fcst_error -- DataFrame showing the new MAPE, stdev, and model for the reallocated forecast
    """
    dataset = data.copy()
    re_testset = dataset[:dataset.index.size - 24].copy()
    fcst_cache = prev_fcst.copy()
    fcst_error = error.copy()
    #mape_new = fcst_cache[:1].copy().reset_index().drop(columns='Month')
    
    #Recalculate testing/training MAPE using a top-down forecast
    trainset, testset, testsize = train_model(re_testset)
    
    fcst_cache['xTotalx'] = fcst_cache.iloc[:,bot_first:bot_last].sum(axis=1)
    
    dataset['xTotalx'] = dataset.iloc[:,bot_first:bot_last].sum(axis=1)
    
    
    if topdown == False:
        #Bottom-up Model
        fcst_cache['xTotalx_AE'] = abs(testset.iloc[:,top_column] - fcst_cache['xTotalx'])
        fcst_cache['xTotalx_APE'] = fcst_cache['xTotalx_AE'] / testset.iloc[:,top_column]
        fcst_error.iloc[0:,top_column] = round(sum(fcst_cache['xTotalx_APE'])/testsize,2)
        fcst_error.iloc[1:,top_column] = round(stdev(fcst_cache['xTotalx_AE']),2)
        fcst_error.iloc[2:,top_column] = "Aggregate"
        
        dataset.iloc[:,top_column] = dataset['xTotalx']
        dataset.drop(dataset.columns[[bot_first,bot_last]], axis=1, inplace = True)
                
    else:
        #Topdown Model
        for i in range(bot_first, bot_last):
            fcst_cache.iloc[:,i] = fcst_cache.iloc[:,i] / fcst_cache['xTotalx'] * fcst_cache.iloc[:,top_column]
            fcst_cache[fcst_cache.columns[i]+'_AE'] = abs(testset.iloc[:,i] - fcst_cache.iloc[:,i])
            fcst_cache[fcst_cache.columns[i]+'_APE'] = fcst_cache[fcst_cache.columns[i]+'_AE'] / testset.iloc[:,i]
            fcst_error.iloc[0,i] = round(sum(fcst_cache[fcst_cache.columns[i]+'_APE'])/testsize,2)
            fcst_error.iloc[1,i] = round(stdev(fcst_cache[fcst_cache.columns[i]+'_AE']),2)
            fcst_error.iloc[2,i] = str(fcst_error.iloc[2,i]) + " Topdown"

        for i in range(bot_first, bot_last):
            dataset.iloc[:,i] = dataset.iloc[:,i] / dataset['xTotalx'] * dataset.iloc[:,top_column]

        dataset.drop(dataset.columns[top_column], axis=1, inplace = True)
    
    dataset.drop(columns='xTotalx', inplace = True)
    fin_data = dataset.copy()
    
    return fin_data, fcst_error

### 3.1 (optional) - Plot Forecast
Builds the final forecast model using the best forecast and complete dataset

In [239]:
def plt_fcst(history, forecast, best_fcst, MAPE, bf_stdev, division):
    """
    Builds the final forecast model using the best forecast and complete dataset
    
    Arguments:
    hours -- DataFrame to be forecasted
    best_fcst -- String of winning forecast model (or override)
    d0 -- Date of final historical month
    
    Returns:
    history -- DataFrame containing all history
    forecast -- DataFrame containing the final forecast
    
    """
    
    forecast['upper'] = forecast.iloc[:,[0]] + (2 * bf_stdev)
    forecast['lower'] = forecast.iloc[:,[0]] - (2 * bf_stdev)

    forecast['upper'] = [0 if i < 0 else i for i in forecast['upper']]
    forecast['lower'] = [0 if i < 0 else i for i in forecast['lower']]
    
    #Plot history and forecast
    plt.figure(figsize=(16,8))
    plt.plot(history, 'g-', label='History')
    plt.plot(forecast.iloc[:,[0]], 'ro--', label=best_fcst+' Forecast')
    
    forecast['upper'].plot(kind='area', style='b:', label='_nolegend_')
    forecast['lower'].plot(kind='area', style = 'w:', label='_nolegend_')
    plt.plot(forecast['lower'], 'b:', label='95% conf')
    
    plt.legend(loc='best')
    plt.axis(ymin=0)
    plt.grid(True)
    plt.axvspan(forecast.index[0], forecast.index.max(), alpha=0.9, color='lightgrey')
    plt.title(division+": Model = "+best_fcst+", MAPE = "+MAPE, fontsize=16)

# Forecast Model
Combines all forecasting models for ease of running

In [240]:
def run_fcst(df_slice,override_fcst=False,override_seasonality=False,plot=True):
    """
    Runs the complete testing,training,forecasting model
    
    Arguments:
    df_slice -- DataFrame to be forecasted
    override_fcst -- String to specify model name for best forecast override (default=False)
    override_seaonality -- String to specify seasonality to override forecast (default=False)
    plot: -- Boolean to specify if the forecast should be plotted (default=True)
    
    Returns:
    history -- DataFrame containing all history
    forecast -- DataFrame containing the final forecast
    fcst_cache -- DataFrame containing the test forecast
    fcst_error -- String containing the forecast MAPE
    bf_stdev -- String containing the standard deviation of errors
    
    """    
    
    #Training Model
    trainset, testset, testsize = train_model(df_slice)
    
    #Forecasting Model
    fcst_results = test_model(trainset, testset, testsize)
    
    #Testing Model
    best_fcst, fcst_error, bf_stdev = acc_model(fcst_results, testsize, override=override_fcst, or_season=override_seasonality)
    
    #Final Forecast
    history, forecast = fcst_model(df_slice, best_fcst, df_slice.index.max())
    
    #Plot Forecast
    if plot == True:
        plt_fcst(history, forecast, best_fcst, "{0:.0f}%".format(fcst_error * 100), bf_stdev, df_slice.columns[0])
    else:
        pass
    
    forecast = forecast.drop(columns=forecast.columns[1:3])
    fcst_cache = fcst_results[best_fcst+'_y_hat'].to_frame().set_index(fcst_results.index)
    fcst_cache.columns=[df_slice.columns[0]]
    
    error = pd.DataFrame({df_slice.columns[0]: [float(fcst_error),round(bf_stdev,2),best_fcst]}, index=['MAPE','stdev','model'])
    
    return history, forecast, fcst_cache, error

# Forecast Run
Runs the forecast models using real data

#### Import and organize data

In [241]:
df = pd.read_excel('Capacity Forecast Data.xlsx', sheet_name='Dataset', parse_dates=['Month'])

# Data organizer
top_df, mid_df, mid_list, mid_cnt, low_dfs, low_lists, low_cnts = data_split(df)

#### Forecast for the top hierarchy

In [ ]:
i = 0
history, forecast, fcst_cache, error = run_fcst(top_df,override_fcst=False,override_seasonality=False,plot=True)

top_fcst = history.append(forecast, sort=True)
top_fcst.columns=['total']

fcst_error = error.copy()

#### Forecasts for the middle hierarchy

In [ ]:
mid_fcst = top_fcst.copy()

for i in range(0, mid_cnt):
    mid = mid_list[i]
    history, forecast, mid_cache, error = run_fcst(mid_df.iloc[:,[i + 1]],override_fcst=False,override_seasonality=False,plot=True)
    
    mid_fcst[mid] = history.append(forecast, sort=True)
    fcst_error[mid] = error
    
    fcst_cache[mid] = mid_cache

In [ ]:
#Test override
#override_name = 'Production Systems'
#override_df = mid_df.loc[:,[override_name]]
#history, forecast, mid_cache, error = run_fcst(override_df,override_fcst='DA',override_seasonality=False,plot=True)

#### Bottom-up reallocation of the middle hierarcy to the top hierarcy (replacing origional forecast)

In [ ]:
new_fcst, error = allocate_fcst(mid_fcst, fcst_cache, fcst_error, 0, 1, mid_cnt, topdown=False)
fcst_hist = new_fcst[:new_fcst.index.size - 24].copy()
fcst_new  = new_fcst[new_fcst.index.size - 24:].copy()

mid_fcst.iloc[:,[0]] = fcst_hist.iloc[:,[0]].append(fcst_new.iloc[:,[0]])
fcst_error = error.copy()

plt_fcst(fcst_hist.iloc[:,[0]], fcst_new.iloc[:,[0]].copy(), fcst_error.iloc[2,[0]].values, "{0:.0f}%".format(float(fcst_error.iloc[0,[0]]) * 100),
         float(fcst_error.iloc[1,[0]]), fcst_error.columns[0])

#### Forecasts for the bottom hierarcy

In [ ]:
#top_df, mid_df, mid_list, mid_cnt, low_dfs, low_lists, low_cnts = data_split(df)
plt.close()

for i in range(0, mid_cnt):
    mid = mid_list[i]
    lows = low_cnts["low_cnt"+str(i)]
    
    for j in range(0, lows):
        low = low_lists["low_list"+str(i)][j]
        low_df = low_dfs["low_df"+str(i)].iloc[:,[j+1]]
        history, forecast, low_cache, error = run_fcst(low_df,override_fcst=False,override_seasonality=False,plot=True)
        
        mid_fcst[low] = history.append(forecast, sort=True)
        fcst_error[low] = error
        fcst_cache[low] = low_cache

In [ ]:
#override_name = 'ov_name'
#override_df = low_dfs["low_dfX"].loc[:,[override_name]]
#history, forecast, low_cache, error = run_fcst(override_df,override_fcst='NA',override_seasonality=False,plot=True)
        
#mid_fcst[override_name] = history.append(forecast, sort=True)
#fcst_error[override_name] = error
#fcst_cache[override_name] = low_cache

#### Top-down reallocation of the middle hierarcy to the bottom hierarcy (replacing origional forecast)

In [ ]:
for i in range(0, mid_cnt):
    mid = mid_list[i]
    lows = low_cnts["low_cnt"+str(i)]
    top_index = i + 1
    top = mid_fcst.columns[top_index]
    
    if lows == 1:
        pass
    else:
        bot_f = mid_fcst.columns.get_loc(low_dfs["low_df"+str(i)].columns[1])
        bot_l = mid_fcst.columns.get_loc(low_dfs["low_df"+str(i)].columns[low_cnts["low_cnt"+str(i)]]) + 1

        new_fcst, error = allocate_fcst(mid_fcst, fcst_cache, fcst_error, top_index, bot_f, bot_l, topdown=True)

        fcst_hist = new_fcst[:new_fcst.index.size - 24].copy()
        fcst_new  = new_fcst[new_fcst.index.size - 24:].copy()

        #is the +1 needed?
        for j in range(bot_f, bot_l):
            mid_fcst.iloc[:,[j]] = fcst_hist.iloc[:,[j-1]].append(fcst_new.iloc[:,[j-1]])
            fcst_error.iloc[:,[j]] = error.iloc[:,[j]]

#### Data Export

In [ ]:
writer = pd.ExcelWriter("Forecast_Export.xlsx", engine='xlsxwriter',datetime_format='MMM-yyyy')

mid_fcst.to_excel(writer,'DataExport')
fcst_error.T.to_excel(writer,'ErrorExport')

writer.save()